In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import numpy as np
from datetime import datetime
from pathlib import Path

# Set style for all visualizations
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

In [ ]:
data_dir = Path("~/Projects/sov-data/data").expanduser()

# Load the dataset
df = pd.read_csv(data_dir / "s&p500-constituents.csv")

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
df.head()

In [ ]:
# Convert dates and handle missing values
df["Date added"] = pd.to_datetime(df["Date added"])
df["Founded"] = pd.to_numeric(df["Founded"], errors="coerce")

# Display missing values
print("Missing values in each column:")
print(df.isnull().sum())

In [ ]:
# Get sector counts
sector_counts = df["GICS Sector"].value_counts()

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Bar plot on the left
sns.barplot(x=sector_counts.values, y=sector_counts.index, ax=ax1)
ax1.set_title("Companies by Sector in S&P 500")
ax1.set_xlabel("Number of Companies")

# Pie chart on the right
ax2.pie(sector_counts.values, labels=sector_counts.index, autopct="%1.1f%%")
ax2.set_title("S&P 500 Sector Composition")
ax2.axis("equal")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Extract states from headquarters location
df["State"] = df["Headquarters Location"].str.extract(r",\s*(\w+)$")

# Get top 15 states
state_counts = df["State"].value_counts().head(15)

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Bar plot on the left
sns.barplot(x=state_counts.values, y=state_counts.index, ax=ax1)
ax1.set_title("Top 15 States by Number of S&P 500 Company Headquarters")
ax1.set_xlabel("Number of Companies")

# Pie chart on the right
ax2.pie(state_counts.values, labels=state_counts.index, autopct="%1.1f%%")
ax2.set_title("Top 15 States Distribution of S&P 500 Headquarters")
ax2.axis("equal")

# Add a custom legend to the pie chart to improve readability
# Reposition it outside the plot
ax2.legend(
    state_counts.index, title="States", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1)
)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Company age distribution
current_year = datetime.now().year
df["Age"] = current_year - df["Founded"]

plt.figure(figsize=(12, 6))
sns.histplot(data=df, x="Age", bins=30)
plt.title("Distribution of Company Ages")
plt.xlabel("Age (Years)")
plt.ylabel("Count")
plt.show()

In [ ]:
# Create custom bins for the timeline
bins = [pd.Timestamp.min] + [pd.Timestamp(f"{year}-01-01") for year in range(2000, 2026, 2)] + [pd.Timestamp.max]
labels = ['Pre-2000'] + [f'{year}-{year+1}' for year in range(2000, 2025, 2)]

# Create a new column with the time periods
df['Time Period'] = pd.cut(df['Date added'], bins=bins, labels=labels)

# Count companies in each period
period_counts = df['Time Period'].value_counts().sort_index()

# Create the plot
plt.figure(figsize=(15, 6))
sns.barplot(x=period_counts.index, y=period_counts.values)
plt.title('S&P 500 Companies by Addition Period')
plt.xlabel('Time Period')
plt.ylabel('Number of Companies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Count the numbers of companies added after 2022
companies_added_after_2022 = df[df['Date added'] > '2022-01-01'].shape[0]
print(f"Number of companies added after 2022: {companies_added_after_2022}")

In [ ]:
# Display oldest companies
print("Oldest Companies in the S&P 500:")
oldest_companies = df.nsmallest(10, "Founded")[["Symbol", "Security", "Founded", "GICS Sector"]]
display(oldest_companies)

# %%
# Display newest companies
print("\nNewest Companies in the S&P 500 (by founding year):")
newest_companies = df.nlargest(10, "Founded")[
    ["Symbol", "Security", "Founded", "GICS Sector"]
]
display(newest_companies)

In [ ]:
# Create summary statistics
summary_stats = pd.DataFrame(
    {
        "Metric": [
            "Total Companies",
            "Number of Sectors",
            "Number of Sub-Industries",
            "Average Company Age",
            "Median Company Age",
            "Oldest Company Year",
            "Newest Company Year",
        ],
        "Value": [
            len(df),
            df["GICS Sector"].nunique(),
            df["GICS Sub-Industry"].nunique(),
            df["Age"].mean().round(1),
            df["Age"].median(),
            df["Founded"].min(),
            df["Founded"].max(),
        ],
    }
)

display(summary_stats)